<a href="https://colab.research.google.com/github/KunihikoK3/DL_RS_GEE/blob/main/GOES16_gif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generation of gif using GOES16 data

Will follow the video [GEE Tutorial #16 - Adding animated text to GIF images](https://www.youtube.com/watch?v=fDnDVuM_Ke4) with corresponding [code](https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/16_add_animated_text.ipynb).

## Initialization

Recall that Colab generates a new virtual machine at each run so nonstandard packages need to be (re)installed.

In [ ]:
!pip install geemap

In [37]:
import geemap
print(geemap.__version__)

0.13.4


In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

### Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [40]:
!ls /content/drive/MyDrive/GEE_data

animation.gif  NOAA_temp.gif  output.gif


## Use geemap to display a gif
I previously downloaded Wu's [example gif](https://github.com/giswqs/geemap/blob/master/examples/data/animation.gif) and placed it in the folder /content/drive/MyDrive/GEE_data

Also let's create file/path names for input and output gifs:

In [22]:
in_gif='/content/drive/MyDrive/GEE_data/animation.gif'
out_gif='/content/drive/MyDrive/GEE_data/output.gif'

In [21]:
geemap.show_image(in_gif)

Output()

### Add animated text to GIF

In [27]:
geemap.add_text_to_gif(
    in_gif,
    out_gif,
    xy=('5%', '5%'),
    text_sequence=1984,
    font_size=30,
    font_color='#0000ff',
    duration=200,
)

In [ ]:
geemap.show_image(out_gif)

### Add place name

In [29]:
geemap.add_text_to_gif(
    out_gif, out_gif, xy=('30%', '85%'), text_sequence="Las Vegas", font_color='black'
)

In [ ]:
geemap.show_image(out_gif)

## Create GIF from Earth Engine data

This example was adapted from the Earth Engine JavaScript API Documentation [here](https://developers.google.com/earth-engine/ic_visualization#video_thumb). 

**NOTE**: Creation of a GIF can be done *without* the use of geemap; see this [notebook](https://gist.github.com/jdbcode/054e8c0c39e052f91db635390a93f710).

In [ ]:
# Define an area of interest geometry with a global non-polar extent.
aoi = ee.Geometry.Polygon(
  [[[-179.0, 78.0], [-179.0, -58.0], [179.0, -58.0], [179.0, 78.0]]], None,
  False)

# Import hourly predicted temperature image collection for northern winter
# solstice. Note that predictions extend for 384 hours; limit the collection
# to the first 24 hours.
temp_col = (ee.ImageCollection('NOAA/GFS0P25')
  .filterDate('2018-12-22', '2018-12-23')
  .limit(24)
  .select('temperature_2m_above_ground'))

# Define arguments for animation function parameters.
video_args = {
  'dimensions': 768,
  'region': aoi,
  'framesPerSecond': 7,
  'crs': 'EPSG:3857',
  'min': -40.0,
  'max': 35.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Get URL that will produce the animation when accessed.
gif_url = temp_col.getVideoThumbURL(video_args)
gif_url

### Download the gif url to Google Drive

In [ ]:
import urllib.request

gif_name = '/content/drive/MyDrive/GEE_data/NOAA_temp.gif'
urllib.request.urlretrieve(gif_url, gif_name)

Use geemap to display

In [ ]:
geemap.show_image(gif_name)

## Create GOES 16 gif
Let's modify the above NOAA gif using the javascript code from [GOES in Earth Engine](https://jstnbraaten.medium.com/goes-in-earth-engine-53fbc8783c16). Wu has a similar [example](https://geemap.org/notebooks/99_landsat_9/) for applying scaling factors to an Image. GEE (javascript) specifications for visualizing GOES 16 images is [here](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GOES_16_MCMIPM?hl=en#image-properties): note similarities with code from [GOES in Earth Engine](https://jstnbraaten.medium.com/goes-in-earth-engine-53fbc8783c16).

### Convert javascript to python
Apply function ```geemap.js_snippet_to_py``` to the javascript code from [GOES in Earth Engine](https://jstnbraaten.medium.com/goes-in-earth-engine-53fbc8783c16). ```geemap.js_snippet_to_py``` does not work in Colab(?) so function was run in VScode. The following code chunks were converted from javascript.

Use a [lambda function](https://gis.stackexchange.com/questions/356588/clipping-a-gee-image-with-python-api) for ImageCollection (did not need after all as .map in

```geosVisCol = ee.ImageCollection('NOAA/GOES/17/MCMIPF').filterDate('2020-09-05T15:00', '2020-09-06T02:00').map(processForVis)```

seems to play the role of the lambda function) and [change notation](https://gis.stackexchange.com/questions/421643/scale-factors-application-not-working-on-gee-pyqgis) for addBands:

In [137]:
def applyScaleAndOffset(img):
  def getFactorImg(factorNames):
    factorList = img.toDictionary().select(factorNames).values()
    return ee.Image.constant(factorList)
  scaleImg = getFactorImg(['CMI_C.._scale'])
  offsetImg = getFactorImg(['CMI_C.._offset'])
  scaled = img.select('CMI_C..').multiply(scaleImg).add(offsetImg)
  #scaled = img.map(lambda image: image.select('CMI_C..').multiply(scaleImg).add(offsetImg)) 
  return img.addBands(srcImg= scaled, overwrite= True)
  # return img.map(lambda image: image.addBands(srcImg = scaled, overwrite = True))

will need [Google Earth Engine Python expression syntax](https://gis.stackexchange.com/questions/419849/google-earth-engine-python-expression-syntax)

In [138]:
# Adds a synthetic green band.
def addGreenBand(img):
  green = img.expression(
    'CMI_GREEN = 0.45 * red + 0.10 * nir + 0.45 * blue',
    {
      'blue': img.select('CMI_C01'),
      'red': img.select('CMI_C02'),
      'nir': img.select('CMI_C03')
    }
  )
  return img.addBands(green)
  # return img.map(lambda image: image.addBands(green))

In [139]:
# Scales select bands for visualization.
def scaleForVis(img):
  return img.select(['CMI_C01', 'CMI_GREEN', 'CMI_C02', 'CMI_C03', 'CMI_C05']) \
              .resample('bicubic') \
              .log10() \
              .interpolate([-1.6, 0.176], [0, 1], 'clamp') \
              .unmask(0) \
              .set('system:time_start', img.get('system:time_start'))

In [140]:
# Wraps previous functions.
def processForVis(img):
  return scaleForVis(addGreenBand(applyScaleAndOffset(img)))

In [141]:
# Get CMI image collection and process it for visualization.
geosVisCol = ee.ImageCollection('NOAA/GOES/17/MCMIPF').filterDate('2020-09-05T15:00', '2020-09-06T02:00').map(processForVis)

# Set display parameters and render the animation.
visParams = {
  'bands': ['CMI_C02', 'CMI_C03', 'CMI_C01'],
  'min': 0,
  'max': 0.8,
  'gamma': 0.8,
  'dimensions': 600,
  'framesPerSecond': 10,
  'region': ee.Geometry.BBox(-123.17, 36.56, -116.22, 40.03),
  'crs': 'EPSG:3857'
};
# print(ui.Thumbnail(geosVisCol, visParams));
# Vis2=processForVis(geosVisCol)
# Get URL that will produce the animation when accessed.
gif_url = geosVisCol.getVideoThumbURL(visParams)
gif_url

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/e85a8252b1c03705931760ccffa9421b-ec3e51f1ec13a264f5508f7a20b3aa79:getPixels'

In [142]:
import urllib.request

gif_name = '/content/drive/MyDrive/GEE_data/GOES.gif'
urllib.request.urlretrieve(gif_url, gif_name)

('/content/drive/MyDrive/GEE_data/GOES.gif',
 <http.client.HTTPMessage at 0x7f1af7fd8950>)

In [143]:
geemap.show_image(gif_name)

Output()